<a href="https://colab.research.google.com/github/joacorapela/svGPFA_simulations/blob/master/bugReport/plotGPUbug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU bug report

## Install code

In [2]:
!rm -rf svGPFA
!git clone -b optimAllParamsSimultaneouslyCUDA https://github.com/joacorapela/svGPFA.git
%pip install git+file:///content/svGPFA
!git clone -b master https://github.com/joacorapela/svGPFA_simulations.git
%cd svGPFA_simulations/bugReport

Cloning into 'svGPFA'...
remote: Enumerating objects: 11760, done.
remote: Counting objects: 100% (2628/2628), done.
remote: Compressing objects: 100% (830/830), done.
remote: Total 11760 (delta 1613), reused 2617 (delta 1610), pack-reused 9132
Receiving objects: 100% (11760/11760), 329.68 MiB | 22.80 MiB/s, done.
Resolving deltas: 100% (7154/7154), done.
Updating files: 100% (300/300), done.
  Cloning file:///content/svGPFA to /tmp/pip-req-build-lkswdctr
  Running command git clone --filter=blob:none --quiet file:///content/svGPFA /tmp/pip-req-build-lkswdctr
  Resolved file:///content/svGPFA to commit 242024ccb6260b8768906648222266696d2c2b91
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


## Import requirements

In [5]:
import os
import traceback
import random
import pickle
import configparser
import torch
import plotly.graph_objects as go

import gcnu_common.utils.config_dict
import svGPFA.stats.svGPFAModelFactory
import svGPFA.stats.svEM
import svGPFA.utils.configUtils
import svGPFA.utils.miscUtils
import svGPFA.utils.initUtils

## Configuration parameters

In [6]:
sim_res_number = 91450833
est_init_number = 551
n_repeats = 1
cuda_device_index = 0
est_init_config_filename_pattern = "../data/{:08d}_estimation_metaData.ini"
sim_res_config_filename_pattern = "../results/{:08d}_simulation_metaData.ini"

# Load data

In [7]:
sim_res_config_filename = sim_res_config_filename_pattern.format(
    sim_res_number)
sim_res_config = configparser.ConfigParser()
sim_res_config.read(sim_res_config_filename)
sim_res_filename = sim_res_config["simulation_results"]["sim_res_filename"]
with open(sim_res_filename, "rb") as f:
    sim_res = pickle.load(f)
spikes_times = sim_res["spikes_times"]
n_trials = len(spikes_times)
n_neurons = len(spikes_times[0])

spikes_times = [[spikes_times[r][n].tolist() for n in range(n_neurons)]
                for r in range(n_trials)]

sim_init_config_filename = sim_res_config["simulation_params"]["sim_init_config_filename"]
sim_init_config = configparser.ConfigParser()
sim_init_config.read(sim_init_config_filename)
trials_start_times = [float(str) for str in sim_init_config["data_structure_params"]["trials_start_times"][1:-1].split(",")]
trials_end_times = [float(str) for str in sim_init_config["data_structure_params"]["trials_end_times"][1:-1].split(",")]

est_init_config_filename = est_init_config_filename_pattern.format(
    est_init_number)
est_init_config = configparser.ConfigParser()
est_init_config.read(est_init_config_filename)
n_latents = int(est_init_config["model_structure_params"]["n_latents"])

## Build parameters

### Build configuration file parameter specifications

In [8]:
args_info = svGPFA.utils.initUtils.getArgsInfo()
strings_dict = gcnu_common.utils.config_dict.GetDict(
    config=est_init_config).get_dict()
config_file_params = svGPFA.utils.initUtils.getParamsDictFromStringsDict(
    n_latents=n_latents, n_trials=n_trials, strings_dict=strings_dict,
    args_info=args_info)

### Build default parameter specificiations

In [9]:
default_params = svGPFA.utils.initUtils.getDefaultParamsDict(
    n_neurons=n_neurons, n_trials=n_trials, n_latents=n_latents)

## Perform estimations

In [10]:
devices = ["cpu", f"cuda:{cuda_device_index}"]
elapsed_times = torch.empty((len(devices), n_repeats), dtype=torch.double)
lower_bounds = torch.empty((len(devices), n_repeats), dtype=torch.double)
num_fun_eval = torch.empty((len(devices), n_repeats), dtype=torch.double)
num_iter = torch.empty((len(devices), n_repeats), dtype=torch.double)

for d, device in enumerate(devices):
    r = 0
    while r < n_repeats:
        print(f"Processing device {device}, repeat {r}")
        #    finally, get the parameters from the dynamic,
        #    configuration file and default parameter specifications
        params, kernels_types = svGPFA.utils.initUtils.getParamsAndKernelsTypes(
            n_trials=n_trials, n_neurons=n_neurons, n_latents=n_latents,
            trials_start_times=trials_start_times,
            trials_end_times=trials_end_times,
            dynamic_params_spec={},
            config_file_params_spec=config_file_params,
            default_params_spec=default_params)

        # build kernels
        kernels_params0 = params["initial_params"]["posterior_on_latents"]["kernels_matrices_store"]["kernels_params0"]
        kernels = svGPFA.utils.miscUtils.buildKernels(
            kernels_types=kernels_types, kernels_params=kernels_params0)

        # create model
        kernelMatrixInvMethod = svGPFA.stats.svGPFAModelFactory.kernelMatrixInvChol
        indPointsCovRep = svGPFA.stats.svGPFAModelFactory.indPointsCovChol
        model = svGPFA.stats.svGPFAModelFactory.SVGPFAModelFactory.buildModelPyTorch(
            conditionalDist=svGPFA.stats.svGPFAModelFactory.PointProcess,
            linkFunction=svGPFA.stats.svGPFAModelFactory.ExponentialLink,
            embeddingType=svGPFA.stats.svGPFAModelFactory.LinearEmbedding,
            kernels=kernels, kernelMatrixInvMethod=kernelMatrixInvMethod,
            indPointsCovRep=indPointsCovRep)

        model.setParamsAndData(
            measurements=spikes_times,
            initial_params=params["initial_params"],
            eLLCalculationParams=params["ell_calculation_params"],
            priorCovRegParam=params["optim_params"]["prior_cov_reg_param"])

        model.to(device=device)

        # maximize lower bound
        svEM = svGPFA.stats.svEM.SVEM_PyTorch()
        try:
            lowerBoundHist, elapsedTimeHist, terminationInfo,\
            iterationsModelParams, num_fun_eval[d, r], num_iter[d, r] = \
                svEM.maximizeSimultaneously(model=model,
                                            optim_params=params["optim_params"])
            elapsed_times[d, r] = elapsedTimeHist[-1]
            lower_bounds[d, r] = lowerBoundHist[-1]
        except Exception as e:
            print("Exception detected. Retrying")
            stack_trace = traceback.format_exc()
            print(e)
            print(stack_trace)
        print("Device {:s}, Repeat {:d}: elapsed time {:f}, "
              "elapsed time per function call {:f}, "
              "lower bound {:f}".format(device, r, elapsed_times[d, r],
                                        elapsed_times[d, r]/num_fun_eval[d, r],
                                        lower_bounds[d, r]))
        r += 1


Processing device cpu, repeat 0
Extracted config_file_params_spec[optim_params][n_quad]=200
Extracted config_file_params_spec[ind_points_locs_params0][n_ind_points]=tensor([9, 9], dtype=torch.int32)
Extracted from config_file c0_distribution=Normal, c0_loc=0.0, c0_scale=1.0, c0_random_seed=None
Extracted from config_file d0_distribution=Normal, d0_loc=0.0, d0_scale=1.0, d0_random_seed=None
Extracted from config_file k_type=exponentialQuadratic and k_lengthsales0=2.0
Extracted from config_file ind_points_locs0_layout=uniform
Extracted from config_file variational_mean0_constant_value=0.0
Extracted from config_file variational_cov0_diag_value=0.1
Extracted config_file_params_spec[optim_params][n_quad]=200
Extracted config_file_params_spec[optim_params][prior_cov_reg_param]=0.001
Extracted config_file_params_spec[optim_params][optim_method]=ECM
Extracted config_file_params_spec[optim_params][em_max_iter]=10
Extracted config_file_params_spec[optim_params][verbose]=True
Extracted config_fil

/usr/local/lib/python3.10/dist-packages/svGPFA/stats/svEM.py:437: UserWarning: posterior variance is larger than the prior variance: k=0, r=4
  warnings.warn(f"posterior variance is larger than the prior variance: k={k}, r={r}")


tensor([ 0.0241,  1.0014,  0.1421,  0.0560, -0.1686,  0.0318,  0.0392,  0.0792,
        -0.0620])
tensor([ 0.0261,  1.0057,  0.1472,  0.0490, -0.1440,  0.0246,  0.0487,  0.0829,
        -0.0595])
tensor([ 0.0237,  1.0021,  0.1458,  0.0162, -0.0527,  0.0057,  0.0608,  0.0816,
        -0.0597])
tensor([ 0.0248,  1.0007,  0.1455,  0.0288, -0.0882,  0.0151,  0.0553,  0.0808,
        -0.0611])
tensor([ 0.0228,  1.0128,  0.1468,  0.0213, -0.0552,  0.0315,  0.0540,  0.0763,
        -0.0649])
tensor([ 0.0219,  1.0880,  0.1615,  0.0213, -0.0416,  0.0323,  0.0573,  0.0810,
        -0.0637])
tensor([ 0.0337,  1.1104,  0.1646,  0.0345, -0.0296,  0.0450,  0.0616,  0.0827,
        -0.0621])
tensor([ 0.0430,  1.1197,  0.1623,  0.0426, -0.0198,  0.0521,  0.0612,  0.0803,
        -0.0658])
tensor([ 5.5185e-02,  1.1370e+00,  1.5758e-01,  4.8827e-02, -1.4579e-04,
         6.1793e-02,  6.0110e-02,  7.5298e-02, -7.4462e-02])
tensor([ 0.0639,  1.1907,  0.1455,  0.0587, -0.0642,  0.0583,  0.0459,  0.0666,
  

/usr/local/lib/python3.10/dist-packages/svGPFA/stats/svEM.py:437: UserWarning: posterior variance is larger than the prior variance: k=1, r=11
  warnings.warn(f"posterior variance is larger than the prior variance: k={k}, r={r}")


tensor([ 0.0762,  1.0003,  0.1626,  0.0856, -0.0820,  0.1299,  0.1359,  0.7089,
        -0.7171])
tensor([ 1.0007,  0.0496,  0.3358,  0.0531, -0.0279, -0.0433,  0.0670,  0.0754,
        -0.1295])
tensor([ 0.0765,  1.0003,  0.1628,  0.0861, -0.0822,  0.1296,  0.1364,  0.7085,
        -0.7178])
tensor([ 1.0010,  0.0493,  0.3365,  0.0535, -0.0279, -0.0414,  0.0673,  0.0754,
        -0.1292])
tensor([ 0.0774,  1.0005,  0.1640,  0.0876, -0.0832,  0.1290,  0.1384,  0.7088,
        -0.7221])
tensor([ 1.0011,  0.1597,  0.0589,  0.0323,  0.0369,  0.0716,  0.0636,  0.1062,
        -0.2719])
tensor([ 1.0037,  0.0492,  0.3394,  0.0546, -0.0278, -0.0390,  0.0684,  0.0753,
        -0.1262])


...

tensor([ 0.0845,  1.0017,  0.1653,  0.0926, -0.0942,  0.1409,  0.1511,  0.6966,
        -0.7562])
tensor([ 1.0211,  0.1624,  0.0607,  0.0327,  0.0372,  0.0712,  0.0659,  0.1095,
        -0.2814])
tensor([ 1.0232,  0.0480,  0.3659,  0.0586, -0.0284, -0.0452,  0.0694,  0.0742,
        -0.1103])
tensor([ 0.0847,  1.0018,  0.1653,  0.0931, -0.0944,  0.1419,  0.1513,  0.6960,
        -0.7573])
tensor([ 1.0006,  0.4663,  0.1335,  0.0834,  0.0778, -0.0905,  0.1002,  0.1540,
        -0.1912])
tensor([ 1.0217,  0.1623,  0.0608,  0.0317,  0.0372,  0.0711,  0.0659,  0.1096,
        -0.2815])
tensor([ 1.0238,  0.0480,  0.3667,  0.0587, -0.0283, -0.0448,  0.0694,  0.0739,
        -0.1102])
tensor([ 0.0846,  1.0018,  0.1649,  0.0934, -0.0947,  0.1430,  0.1514,  0.6943,
        -0.7576])
tensor([ 1.0010,  0.4665,  0.1342,  0.0839,  0.0782, -0.0898,  0.1003,  0.1545,
        -0.1919])
tensor([ 1.0216,  0.1621,  0.0608,  0.0310,  0.0372,  0.0709,  0.0661,  0.1091,
        -0.2814])
tensor([ 1.0238,  0.

/usr/local/lib/python3.10/dist-packages/svGPFA/stats/svEM.py:437: UserWarning: posterior variance is larger than the prior variance: k=0, r=3
  warnings.warn(f"posterior variance is larger than the prior variance: k={k}, r={r}")
/usr/local/lib/python3.10/dist-packages/svGPFA/stats/svEM.py:437: UserWarning: posterior variance is larger than the prior variance: k=0, r=10
  warnings.warn(f"posterior variance is larger than the prior variance: k={k}, r={r}")
/usr/local/lib/python3.10/dist-packages/svGPFA/stats/svEM.py:437: UserWarning: posterior variance is larger than the prior variance: k=1, r=0
  warnings.warn(f"posterior variance is larger than the prior variance: k={k}, r={r}")
/usr/local/lib/python3.10/dist-packages/svGPFA/stats/svEM.py:437: UserWarning: posterior variance is larger than the prior variance: k=1, r=2
  warnings.warn(f"posterior variance is larger than the prior variance: k={k}, r={r}")
/usr/local/lib/python3.10/dist-packages/svGPFA/stats/svEM.py:437: UserWarning: post

tensor([ -3.1731, -11.5290,  -0.5974,  -0.3616,  -1.2680,  -0.8031,   0.5456,
          1.2785, -10.2772])
tensor([ -4.0559,  -2.1081,  -2.5511,  -2.5336,  -1.3566,   1.5916,   1.3444,
         -5.5035, -13.7062])
tensor([-3.9930, -3.4609, -3.5471, -1.1854, -0.1290,  0.8989,  3.0627, -6.2353,
        -9.8162])
tensor([ -3.2808,  -3.6389,  -1.4902,   2.1301,  -2.7706,  -4.3068,  -7.9295,
         -3.9834, -14.8402])
tensor([-2.7620e+00, -7.1196e+00, -6.7501e-01, -3.3577e-01,  1.0676e+00,
         6.9134e-03, -2.7743e+00, -3.9145e+00, -8.7811e+00])
tensor([-4.4414, -1.3509, -0.5624, -0.9070, -0.6003,  0.8699,  3.1422,  2.2023,
        -9.4775])
tensor([ -6.1990,  -1.1045,  -2.6053,  -2.5222,  -0.9180,   2.5643,  -2.2506,
         -6.2926, -12.4073])
tensor([-2.6274, -1.3899, -1.6873, -1.6781, -0.8953,  1.0688,  0.9047, -3.6427,
        -9.1175])
tensor([-2.5827, -2.2878, -2.3521, -0.7796, -0.0770,  0.6078,  2.0498, -4.1300,
        -6.5329])
tensor([-2.1494, -2.4134, -0.9845,  1.4279, -1

/usr/local/lib/python3.10/dist-packages/svGPFA/stats/svEM.py:437: UserWarning: posterior variance is larger than the prior variance: k=1, r=9
  warnings.warn(f"posterior variance is larger than the prior variance: k={k}, r={r}")


tensor([-0.0820,  1.0062, -0.2848, -0.2407, -0.1919, -0.1537, -0.1773, -0.1555,
        -0.9315])
tensor([-0.0329,  1.1703, -0.1751, -0.0454, -0.1634, -0.1736, -0.0548, -0.0682,
        -1.2810])
tensor([-0.0181,  1.0999, -0.1621, -0.0499, -0.1465, -0.1612, -0.0518, -0.0665,
        -1.2074])
tensor([-0.0033,  1.0302, -0.1492, -0.0544, -0.1298, -0.1489, -0.0488, -0.0648,
        -1.1345])
tensor([-1.3937e-04,  1.0022e+00, -1.5192e-01, -6.3845e-02, -1.2794e-01,
        -1.4921e-01, -5.2902e-02, -7.1156e-02, -1.1201e+00])
tensor([-0.0570,  1.1505, -0.2116, -0.0998, -0.1842, -0.2053, -0.0833, -0.1184,
        -1.3279])
tensor([-0.0308,  1.0820, -0.1840, -0.0832, -0.1582, -0.1794, -0.0693, -0.0966,
        -1.2319])
tensor([-0.0124,  1.0341, -0.1648, -0.0716, -0.1400, -0.1613, -0.0594, -0.0813,
        -1.1648])
Iteration 01, end: 9077.293045, niter: 68, nfeval: 126
Iteration 02, start: 9077.293045
tensor([-0.0124,  1.0341, -0.1648, -0.0716, -0.1400, -0.1613, -0.0594, -0.0813,
        -1.1

## Plot results

In [11]:
hoovertext = [[],[]]
for d, device in enumerate(devices):
    for r in range(n_repeats):
        hoovertext[d].append(
            (f"num_fun_eval: {num_fun_eval[d,r]}<br>"
             f"num_iter: {num_iter[d,r]}<br>"
             f"elapsed_time_per_funcall: "
             f"{elapsed_times[d,r]/num_fun_eval[d,r]}"))

fig = go.Figure()
for d in range(len(devices)):
    trace = go.Scatter(x=elapsed_times[d,:], y=lower_bounds[d, :],
                       mode="markers", name=devices[d],
                       hovertext=hoovertext[d], hoverinfo="text")
    fig.add_trace(trace)
fig.update_layout(xaxis_title="Elapsed Time (sec)")
fig.update_layout(yaxis_title="Lower Bound")

fig